In [ ]:
!nvidia-smi


Sun Aug 15 20:30:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------




---




## 將 Google Drive 掛載到 Colab 目錄下

掛載 Google Drive 的好處是不用每次都手動上傳或下載檔案，而且還能讓訓練好的模型檔案自動保存到 Google Drive。這樣就不會因為 Colab 中斷後就必須從頭訓練。

In [ ]:
# 將 Google Drive 掛載到 Colab 目錄下
from google.colab import drive,files 
drive.mount('/drive', force_remount=True)
#直接本地端上傳檔案
#uploaded = files.upload()

Mounted at /drive


# 準備資料集

您可以使用自己的資料夾。為了示範，這邊使用 [The Oxford-IIIT Pet Dataset](http://www.robots.ox.ac.uk/~vgg/data/pets/) 已經標注好的寵物資料來當作範例。我們會用到裡面的圖片檔以及 `annotations` 中的 `xml` 標註資料。

從雲端硬碟下載data
將data存放到splend_detection


In [ ]:

!cp -r /drive/My\ Drive/專題製作/model_taring_yolo/splend_detection /content/
#%rm -rf <folder_name>


## 定義後來會用到的檔案路徑

路徑分為保存在 Colab 的，每次 Colab 被重置後就會消失。所以只放一些不重要的東西。
保存在 Google Drive 的檔案就是會被保存下來的，即使 Colab 被重置後，也不會消失；我們會把重要的東西存在這，例如設定檔、訓練到一半的模型等。

In [ ]:
# define constants

LOCAL_IMAGES_DIR_PATH = "/content/splend_detection/IMG"
LOCAL_LABELS_DIR_PATH = "/content/splend_detection/LABEL"
LOCAL_YOLOS_DIR_PATH = "/content/splend_detection/yolos"
LOCAL_CFG_DIR_PATH = "/content/splend_detection/cfg"

GDRIVE_APP_BASE_DIR_REMOTE_PATH = "/drive/My\ Drive/專題製作/model_taring_yolo/train_yolo_with_custom_dataset_on_colab_101"

GDRIVE_APP_BASE_DIR_PATH = "/content/app"
GDRIVE_WEIGHTS_DIR_PATH = GDRIVE_APP_BASE_DIR_PATH+"/weights"
GDRIVE_CFG_DIR_PATH = GDRIVE_APP_BASE_DIR_PATH+"/cfg"
GITHUB_CODEBASE_DIR_PATH = "/content/train_yolo_with_custom_dataset_on_colab_101"

GDRIVE_DARKNET_BIN_FILE_PATH = GITHUB_CODEBASE_DIR_PATH+"/darknet"


In [ ]:
# load sample codes
%cd /content
!git clone https://github.com/wallat/train_yolo_with_custom_dataset_on_colab_101.git



/content
Cloning into 'train_yolo_with_custom_dataset_on_colab_101'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 23 (delta 4), reused 19 (delta 3), pack-reused 0
Unpacking objects: 100% (23/23), done.


In [ ]:
# build the link to avoiding type in long path name everytime
!ln -fs {GDRIVE_APP_BASE_DIR_REMOTE_PATH} {GDRIVE_APP_BASE_DIR_PATH}

In [ ]:
# clean folders

import os
import shutil

shutil.rmtree(LOCAL_CFG_DIR_PATH, ignore_errors=True)
shutil.rmtree(LOCAL_YOLOS_DIR_PATH, ignore_errors=True)

#create folders
os.makedirs(GDRIVE_APP_BASE_DIR_REMOTE_PATH.replace("\ ", " "), exist_ok=True)
os.makedirs(GDRIVE_CFG_DIR_PATH, exist_ok=True)
os.makedirs(GDRIVE_WEIGHTS_DIR_PATH, exist_ok=True)

os.makedirs(LOCAL_CFG_DIR_PATH, exist_ok=True)
os.makedirs(LOCAL_YOLOS_DIR_PATH, exist_ok=True)
# os.makedirs(GDRIVE_CFG_DIR_PATH, exist_ok=True)



In [ ]:
# Convert VOC xmls into Yolo's format

import glob
import os
import re

labels = set()
for path in glob.glob(os.path.join(LOCAL_LABELS_DIR_PATH, "*.xml")):
    with open(path, 'r') as f:
        content = f.read()

    # extract label names
    matches = re.findall(r'<name>([\w_]+)<\/name>', content, flags=0)
    labels.update(matches)

# write label into file`
with open(os.path.join(GDRIVE_CFG_DIR_PATH, "obj.names"), 'w') as f:
    f.write("\n".join(labels))

print('Read in %d labels: %s' % (len(labels), ", ".join(labels)))

Read in 5 labels: 4, 1, 2, 5, 3


## 將 VOC 格式的標記資料轉成 YOLO 的標記資料

Yolo 不是使用標準的格式，原本的 VOC 標記格式需要轉換後才能使用在 darkent 框架上。
這邊就不詳細解釋如何轉換，對如何轉換的詳細規格可以參考 Yolo 官網 。我們直接使用我從 convert2Yolo 套件中擷取出來的片段程式碼來執行轉換，並把轉換的結果都放到 `/content/splend_detection/yolos` 目錄中。

In [ ]:
import sys
sys.path.append(GITHUB_CODEBASE_DIR_PATH)

from Format import VOC, YOLO

voc = VOC()
yolo = YOLO(os.path.join(GDRIVE_CFG_DIR_PATH, "obj.names"))

flag, data = voc.parse(LOCAL_LABELS_DIR_PATH)
flag, data = yolo.generate(data)

flag, data = yolo.save(data,
    save_path=LOCAL_YOLOS_DIR_PATH,
    img_path=LOCAL_IMAGES_DIR_PATH, img_type=".jpg", manipast_path="./")

l ['4', '1', '2', '5', '3']

VOC Parsing:   |████████████████████████████████████████| 100.0% (475/475)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (475/475)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (475/475)  Complete



In [ ]:
# copy images into yolos folder
# !find $LOCAL_IMAGES_DIR_PATH -name "*.jpg" -exec cp {} /content/splend_detection/yolos \;

# from distutils.dir_util import copy_tree
# copy_tree(LOCAL_IMAGES_DIR_PATH, LOCAL_YOLOS_DIR_PATH)

!cp {LOCAL_IMAGES_DIR_PATH}/*.jpg {LOCAL_YOLOS_DIR_PATH}

## 準備訓練用的設定檔

* `obj.names`：所有的物體標籤名稱，每一行一個。例如此例中只會有兩行，分別是 cat dog 。
* `yolov3.cfg`：darknet 網路的設定檔，描述每一層網路應該要如何建立，以及建立多少 node 等。裡面有些數值需要根據你的訓練資料來個別設定。
* `train.txt` `test.txt` ：這兩個檔案告訴 darknet 要到哪個路徑下找到訓練用的圖片。
* `obj.data`：darknet 的主要設定檔案，告訴 darknet 其他的設定檔路徑。darknet 會一一去讀取其他的檔案。

In [ ]:
# create the cfg file
!cp {GITHUB_CODEBASE_DIR_PATH}/darknet_cfg/yolov3.cfg {GDRIVE_CFG_DIR_PATH}/yolov3.cfg

# fetch label_names
with open(os.path.join(GDRIVE_CFG_DIR_PATH, "obj.names"), 'r') as f:
  f_content = f.read()
label_names = f_content.strip().splitlines()

# update the cfg file
with open(os.path.join(GDRIVE_CFG_DIR_PATH, "yolov3.cfg"), 'r') as f:
  content = f.read()
with open(os.path.join(GDRIVE_CFG_DIR_PATH, "yolov3.cfg"), 'w') as f:
  num_max_batches = len(label_names)*2000
  content = content.replace("%NUM_CLASSES%", str(len(label_names)))
  content = content.replace("%NUM_MAX_BATCHES%", str(num_max_batches))
  content = content.replace("%NUM_MAX_BATCHES_80%", str(int(num_max_batches*0.8)))
  content = content.replace("%NUM_MAX_BATCHES_90%", str(int(num_max_batches*0.9)))
  content = content.replace("%NUM_CONVOLUTIONAL_FILTERS%", str((len(label_names)+5)*3))

  f.write(content)

修改yolov3.cfg裡的filters=(classes+5)*3，classes=5
行數603 610 689 696 776 783


18行learning_rate=0.001學習率建議0.01-0.001，剛開始學習大膽學習，可調整為0.01，對資料比較熟悉後，在調整為0.001


In [ ]:
# create train and test files
import random
import glob

txt_paths = glob.glob(os.path.join(LOCAL_YOLOS_DIR_PATH, "*.txt"))

random.shuffle(txt_paths)
num_train_images = int(len(txt_paths)*0.8)

#check file path exist
assert num_train_images>0, "There's no training images in folder %s" % (LOCAL_YOLOS_DIR_PATH)

with open(os.path.join(GDRIVE_CFG_DIR_PATH, "train.txt"), 'w') as f:
  for path in txt_paths[:num_train_images]:
    f.write("%s/%s\n" % (LOCAL_YOLOS_DIR_PATH, os.path.basename(path).replace(".txt", ".jpg")))
with open(os.path.join(GDRIVE_CFG_DIR_PATH, "test.txt"), 'w') as f:
  for path in txt_paths[num_train_images:]:
    f.write("%s/%s\n" % (LOCAL_YOLOS_DIR_PATH, os.path.basename(path).replace(".txt", ".jpg")))

# create obj
with open(os.path.join(GDRIVE_CFG_DIR_PATH, "obj.data"), 'w') as f:
  f.write("classes=%d\n" % (len(label_names)))
  f.write("train=%s/train.txt\n" % (GDRIVE_CFG_DIR_PATH))
  f.write("valid=%s/test.txt\n" % (GDRIVE_CFG_DIR_PATH))
  f.write("names=%s/obj.names\n" % (GDRIVE_CFG_DIR_PATH))
  f.write("backup=%s\n" % (GDRIVE_WEIGHTS_DIR_PATH))

## 準備 darkent 執行檔

我們直接從之前已經編譯好的檔案複製過來就好，不用每次都重頭編譯那實在是太~花~時~間~了~。編譯的方法請見 「[如何在 Colab 安裝 Darknet 框架訓練 YOLO v3 物件辨識並且最佳化 Colab 的訓練流程](http://bit.ly/33XjcEu)」這邊文章。

In [ ]:
# copy the pretrained darknet bin file
!cp {GDRIVE_DARKNET_BIN_FILE_PATH} /content/
assert os.path.isfile("/content/darknet"), 'Cannot copy from %s to /content' % (GDRIVE_DARKNET_BIN_FILE_PATH)

!chmod +x /content/darknet

## （可選）使用 darknet 預先訓練的基底模型

Darknet 也好心的提供了預先訓練的模型，以此為基底，可以讓後來的訓練比較快達到較好的辨識率。但前提是你的圖片都是常見的圖片，例如一般照片、場景照片等；如果是一些遊戲畫面很少見的，從 0 開始訓練可能會達到比較好的效果。

In [ ]:
# Use the pre-trained weights to speed up the training speed
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-07-19 00:01:49--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   107MB/s    in 1.4s    

2021-07-19 00:01:51 (107 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



# 終於可以開始訓練模型了～

上面的步驟看起來很多，但其實只要寫好一次，之後每次訓練時只要換上自己要的資料夾，然後按 `Run all` 就可以了。

這邊提供兩條指令：

1. 是從 YOLO Pre-trained model 開始訓練，如果你的資料集是一般照片，那用這個效果會比較好。

2. 是從上次訓練的中斷點繼續往下訓練；指令會讀取資料夾內的 weight 檔案，從上次中斷點繼續訓練下去。適合從 0 開始訓練或是被 Colab 中斷後重新開始訓練。

每個batch都會有這樣一個輸出：

2706: 1.350835, 1.386559 avg, 0.001000 rate, 3.323842 seconds, 173184 images

2706：batch是第幾組。

1.350835：總損失

1.386559 avg ： 平均損失，应该越低越好, 一般来说, 一旦这个数值低于 0.060730 avg 就可以终止训练了;

0.001000 rate：目前的學習率

3.323842 seconds： 當前batch訓練所花的時間

173184 images ： 目前為止參與訓練的圖片總數 = 2706 * 64

Region 82 Avg IOU: 0.798032, Class: 0.559781, Obj: 0.515851, No Obj: 0.006533, .5R: 1.000000, .75R: 1.000000,  count: 2

Region Avg IOU:表示在當前subdivision內的圖片的平均IOU，代表預測的矩形框和真實目
標的交集與並集之比.

Class:標註物體分類的正確率，期望該值趨近於1。

Obj:  越接近1越好。

No Obj: 期望該值越來越小，但不為零。

 
count: count後的值是所有的當前subdivision圖片（本例中一共4張）中包含正樣本的圖片的數量。

In [ ]:
# train the model

#!./darknet detector train {GDRIVE_CFG_DIR_PATH}/obj.data {GDRIVE_CFG_DIR_PATH}/yolov3.cfg darknet53.conv.74 
!./darknet detector train {GDRIVE_CFG_DIR_PATH}/obj.data {GDRIVE_CFG_DIR_PATH}/yolov3.cfg darknet53.conv.74  -dont_show | grep "avg loss"
#!./darknet detector train {GDRIVE_CFG_DIR_PATH}/obj.data {GDRIVE_CFG_DIR_PATH}/yolov3.cfg {GDRIVE_WEIGHTS_DIR_PATH}/yolov3_last.weights  -dont_show | grep "avg loss"



layer     filters    size              input                output
   0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32 0.299 BF
   1 conv     64  3 x 3 / 2   416 x 416 x  32   ->   208 x 208 x  64 1.595 BF
   2 conv     32  1 x 1 / 1   208 x 208 x  64   ->   208 x 208 x  32 0.177 BF
   3 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1
   5 conv    128  3 x 3 / 2   208 x 208 x  64   ->   104 x 104 x 128 1.595 BF
   6 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64 0.177 BF
   7 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5
   9 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64 0.177 BF
  10 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128 1.595 BF
  11 Shortcut Layer: 8
  12 conv    256  3 x 3 / 2   104 x 104 x 128   ->    52 x  52 x 256 1.595 BF
  13 conv    128  1 x 1 / 1    52 x  52 x 256   ->    52 x  52 x 128 0.177 BF
  14 c

1# 檢視訓練成果

檢測圖片時，我們只需要 `obj.names`, `yolov3.cfg` 以及 weights 檔就夠。我們可以直接利用 Opencv 內建的 darknet 來讀取網路並產生出預測。

In [ ]:

!cp -r  /drive/My\ Drive/專題製作/splend_detection/test/  /content/splend_detection/cfg/


cp: cannot stat '/drive/My Drive/專題製作/splend_detection/test/': No such file or directory


In [ ]:
#TO_DETECTING_IMAGE_DIR_PATH = GITHUB_CODEBASE_DIR_PATH+"/to_detect_images"
TO_DETECTING_IMAGE_DIR_PATH = "/content/splend_detection/test"

# Use python to read 

import os
import cv2
import numpy as np
import glob
from google.colab.patches import cv2_imshow

import pprint
pp = pprint.PrettyPrinter(indent=4)


def detecting_one_image(net, output_layers, img):
  # Detecting objects
  # cv::dnn::blobFromImage (InputArray image, double scalefactor=1.0, const Size &size=Size(), const Scalar &mean=Scalar(), bool swapRB=false, bool crop=false, int ddepth=CV_32F)
  #对图像进行预处理，包括减均值，比例缩放，裁剪，交换通道等，返回一个4通道的blob(blob可以简单理解为一个N维的数组，用于神经网络的输入)
  blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
  net.setInput(blob)
  outs = net.forward(output_layers)

  return outs

# Load Yolo
net = cv2.dnn.readNet(GDRIVE_WEIGHTS_DIR_PATH+"/0704/yolov3_0704_3500.weights", GDRIVE_CFG_DIR_PATH+"/0704/yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Load label names
with open(GDRIVE_CFG_DIR_PATH+"/0704/obj.names", "r") as f:
  classes = [line.strip() for line in f.readlines()]

# Generate display colors
colors = np.random.uniform(0, 255, size=(len(classes), 3))

for fpath in glob.glob(os.path.join(TO_DETECTING_IMAGE_DIR_PATH, "*.jpg")):
  print("fpath", fpath)

  # Loading image
  img = cv2.imread(fpath)
  height, width, channels = img.shape

  if width>1000: # resize for display purpose
    dim = (1000, int(1000*height/width))
    img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
    height, width, channels = img.shape

  outs = detecting_one_image(net, output_layers, img)

  # Showing informations on the screen
  for out in outs:
    for detection in out:
      scores = detection[5:]
      class_id = np.argmax(scores)
      confidence = scores[class_id]
      if confidence > 0.3:
        # Object detected
        center_x = int(detection[0] * width)
        center_y = int(detection[1] * height)
        w = int(detection[2] * width)
        h = int(detection[3] * height)

        # Rectangle coordinates
        x = int(center_x - w / 2)
        y = int(center_y - h / 2)

        label = "(%.2f) %s" % (confidence, classes[class_id])

        cv2.rectangle(img, (x, y), (x + w, y + h), colors[class_id], 2)#在檢測目標上畫矩形框
        cv2.putText(img, label, (x, y+h-5), cv2.FONT_HERSHEY_PLAIN, 1, colors[class_id], 1)

  cv2_imshow(img)

NameError: ignored